In [2]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.interpolate import splprep, splev

df_piezometres = pd.DataFrame({
    'Forage': ['S1', 'S2', 'S3'],
    'X': [988783.98, 988995.89, 989105.07],
    'Y': [6572463.58, 6572431.00, 6572511.10],
    'Z_surface': [1154.28, 1164.32, 1180.61],
    'Niveau_nappe': [1146.125, 1157.385, 1169.895],  
    'Profondeur_piezo': [1145.725, 1153.935, 1169.895]  
})

df_riviere = pd.read_csv('Altimétrie dranse.csv', sep=';')

tck, u = splprep([df_riviere['X'], df_riviere['Y'], df_riviere['Z']], s=0.01)
u_new = np.linspace(u.min(), u.max(), 30)
x_riviere_lisse, y_riviere_lisse, z_riviere_lisse = splev(u_new, tck)

x_min, x_max = min(df_piezometres['X'].min(), df_riviere['X'].min()), max(df_piezometres['X'].max(), df_riviere['X'].max())
y_min, y_max = min(df_piezometres['Y'].min(), df_riviere['Y'].min()), max(df_piezometres['Y'].max(), df_riviere['Y'].max())
x_grid = np.linspace(x_min, x_max, 100)
y_grid = np.linspace(y_min, y_max, 100)
x_grid, y_grid = np.meshgrid(x_grid, y_grid)

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=x_riviere_lisse, y=y_riviere_lisse, z=z_riviere_lisse, mode='lines',
                           line=dict(color='dodgerblue', width=8), name='La Dranse'))

for _, row in df_piezometres.iterrows():
    x_piezometer = row['X']
    y_piezometer = row['Y']
    z_surface = row['Z_surface']
    z_nappe = row['Niveau_nappe']
    z_piezometer_depth = row['Profondeur_piezo']
    
    fig.add_trace(go.Scatter3d(x=[x_piezometer], y=[y_piezometer], z=[z_surface],
                               mode='markers', marker=dict(size=11, color='white'), name=row['Forage']))

    fig.add_trace(go.Scatter3d(x=[x_piezometer, x_piezometer], y=[y_piezometer, y_piezometer],
                               z=[z_surface, z_nappe], mode='lines',
                               line=dict(color='grey', width=8), name=row['Forage']))

    fig.add_trace(go.Scatter3d(x=[x_piezometer, x_piezometer], y=[y_piezometer, y_piezometer],
                               z=[z_nappe, z_piezometer_depth], mode='lines',
                               line=dict(color='blue', width=8), name=row['Forage'] + ' Niveau piézométrique'))

    fig.add_trace(go.Scatter3d(x=[x_piezometer], y=[y_piezometer], z=[z_surface],
                               text=[row['Forage']], mode='text', textposition='middle center',
                               textfont=dict(size=15), name=row['Forage'] + ' Nom'))

for z_level in df_piezometres['Niveau_nappe']:
    z_grid = np.full_like(x_grid, z_level)  
    
    surface = go.Surface(x=x_grid, y=y_grid, z=z_grid, colorscale='Viridis_R', showscale=False, opacity=0.4)
    fig.add_trace(surface)

fig.update_layout(scene=dict(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_title='Altitude (Z)'),
                  title='Niveau piézometrique des forages par rapport à la Dranse le 29/03/2023',
                  margin=dict(l=0, r=0, b=0, t=40))

fig.show()
